In [1]:
!git clone https://github.com/recommenders-team/recommenders.git


Cloning into 'recommenders'...
remote: Enumerating objects: 43247, done.
remote: Counting objects: 100% (1718/1718), done.
remote: Compressing objects: 100% (673/673), done.
remote: Total 43247 (delta 1298), reused 1072 (delta 1044), pack-reused 41529 (from 4)
Receiving objects: 100% (43247/43247), 223.23 MiB | 32.51 MiB/s, done.
Resolving deltas: 100% (28934/28934), done.


In [2]:
%pip install retrying

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.15.0

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.

In [4]:
import os
import sys
import numpy as np
import zipfile
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
import sys

sys.path.append('/kaggle/working/recommenders/')

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Tensorflow version: 2.15.0


In [5]:
epochs = 5
seed = 42
batch_size = 32

In [6]:
train_news_file = "/kaggle/input/amazon-3/processed_data/train/news.tsv"
train_behaviors_file = "/kaggle/input/amazon-3/processed_data/train/behaviors.tsv"
valid_news_file = "/kaggle/input/amazon-3/processed_data/valid/news.tsv"
valid_behaviors_file = "/kaggle/input/amazon-3/processed_data/valid/behaviors.tsv"
wordEmb_file = "/kaggle/input/amazon-3/processed_data/utils/embedding_all_300d.npy"
userDict_file = "/kaggle/input/amazon-3/processed_data/utils/uid2index.pkl"
wordDict_file = "/kaggle/input/amazon-3/processed_data/utils/word_dict_all.pkl"
vertDict_file = "/kaggle/input/amazon-3/processed_data/utils/vert_dict.pkl"
subvertDict_file = "/kaggle/input/amazon-3/processed_data/utils/subvert_dict.pkl"
yaml_file = "/kaggle/input/amazon-3/processed_data/naml.yaml"

In [2]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file, 
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)


HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 100000, 'title_size': 30, 'body_size': 50, 'his_size': 50, 'vert_num': 2, 'subvert_num': 673, 'data_format': 'naml', 'npratio': 4, 'metrics': ['ndcg@5;10', 'hit@10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'naml', 'dense_activation': 'relu', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/kaggle/input/amazon-3/processed_data/utils/embedding_all_300d.npy', 'wordDict_file': '/kaggle/input/amazon-3/processed_data/utils/word_dict_all.pkl', 'userDict_file': '/kaggle/input/amazon-3/processed_data/utils/uid2index.pkl', 'vertDict_file': '/kaggle/input/amazon-3/processed_data/utils/vert_dict.pkl', 'subvertDict_file': '/

In [8]:
iterator = MINDAllIterator

In [9]:
model = NAMLModel(hparams, iterator, seed=seed)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [1]:
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically. updates=self.state_updates,
312it [01:44, 3.08it/s]
9969it [02:09, 77.0it/s]
1862143it [03:21:31, 154it/s]


at epoch 1
train info: logloss loss:1.330520157244
eval info: ndcg@10: 0.4895, ndcg@5: 0.3895, hit@10: 0.5895
at epoch 1


312it [01:30, 3.56it/s]
9969it [01:52, 89.0it/s]
1862143it [02:54:21, 178it/s]


at epoch 2
train info: logloss loss:1.188281057155
eval info: ndcg@10: 0.5517, ndcg@5: 0.4517, hit@10: 0.6517
at epoch 2


312it [01:12, 4.4it/s]
9969it [01:30, 110.0it/s]
1862143it [02:21:04, 220it/s]


at epoch 3
train info: logloss loss:1.017078280396
eval info: ndcg@10: 0.6429, ndcg@5: 0.5429, hit@10: 0.7429
at epoch 3


312it [01:33, 3.44it/s]
9969it [01:55, 86.0it/s]
1862143it [03:00:26, 172it/s]


at epoch 4
train info: logloss loss:0.881351284752
eval info: ndcg@10: 0.6986, ndcg@5: 0.5986, hit@10: 0.7986
at epoch 4


312it [01:26, 3.7it/s]
9969it [01:47, 92.5it/s]
1862143it [02:47:45, 185it/s]


at epoch 5
train info: logloss loss:0.756695607530
eval info: ndcg@10: 0.7433, ndcg@5: 0.6433, hit@10: 0.8433
at epoch 5


In [14]:
# # Record results for tests - ignore this cell
# store_metadata("ndcg@5", res_syn['ndcg@5'])
# store_metadata("ndcg@10", res_syn['ndcg@10'])

In [15]:
model_path = "/kaggle/working/model"
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "naml_ckpt"))
